In [1]:
import pickle

from anngel.datalib.s3datastore import S3DataStore
from anngel.datasets.openbiolink import OpenBioLinkDataset
from anngel.datasets.custom_dataset import CustomDataset

import pandas as pd
import numpy as np
import pandas as pd

from pathlib import Path

out_dir = Path("tmp")
out_dir.mkdir(exist_ok=True)

ds = OpenBioLinkDataset(out_dir / "openbiolink")
TRAIN_IDS_FILE = "../../data/editedOpenBioLink/train_genes_id.csv"
VAL_IDS_FILE = "../../data/editedOpenBioLink/val_genes_id.csv"
TEST_IDS_FILE = "../../data/editedOpenBioLink/test_genes_id.csv"

irr_node = "HP:0000855"

edges = ds.get_edges()
nodes = ds.get_nodes()
genes = nodes.query('node_type == "GENE"')["node_id"].values
small_irr_genes = edges.query(
    "target_node_id == @irr_node and source_node_id.isin(@genes)"
)["source_node_id"].values

large_irr_genes = (
    pd.concat(
        [
            pd.read_csv(TRAIN_IDS_FILE, index_col=0),
            # pd.read_csv(VAL_IDS_FILE, index_col=0),
            # pd.read_csv(TEST_IDS_FILE, index_col=0),
        ]
    )
    .iloc[:, 0]
    .values
)

missing_irr_genes = large_irr_genes[~np.isin(large_irr_genes, small_irr_genes)]

new_edges = pd.DataFrame.from_records(
    [
        (gene, "GENE_PHENOTYPE", irr_node, np.NaN, "CUSTOM")
        for gene in missing_irr_genes
    ],
    columns=edges.columns,
)

edges_enriched = pd.concat((edges, new_edges))
nodes_enriched = nodes

ds_name = "enriched_openbiolink"
(out_dir / ds_name).mkdir(exist_ok=True, parents=True)
edges_enriched.to_csv(out_dir / ds_name / "edges.tsv", index=False, sep="\t")
nodes_enriched.to_csv(out_dir / ds_name / "nodes.tsv", index=False, sep="\t")
ds = CustomDataset(out_dir, ds_name)

Found 12 files in local directory "tmp/s3_files".


In [2]:
graph = ds.to_networkx()
with open("graph_obl_enriched_directed.pkl", "wb") as f:
    pickle.dump(graph, f)
with open("graph_obl_enriched_undirected.pkl", "wb") as f:
    pickle.dump(graph.to_undirected(), f)
del graph

genes = ds.get_nodes().query('node_type == "GENE"')["node_id"].values
irr_genes = (
    ds.get_edges()
    .query("target_node_id == @irr_node and source_node_id.isin(@genes)")[
        "source_node_id"
    ]
    .values
)
df = ds.get_nodes()
df["irr_gene"] = False
df.loc[df["node_id"].isin(irr_genes), "irr_gene"] = True
df

irr_diseases_df = pd.read_csv(
    "../../data/IR_related_diseases_DOID_533.tsv", sep="\t", header=None
)
irr_diseases_df.columns = ["Disease", "URL"]
irr_diseases_df["node_id"] = (
    irr_diseases_df["URL"].str.split("/").apply(lambda r: r[-1].replace("_", ":"))
)
irr_diseases_df.to_csv("ir_related_diseases.csv", index=False)
df["irr_disease"] = df["node_id"].isin(irr_diseases_df["node_id"])

irr_pathways_df = pd.read_csv("../../data/IRrelatedPathways", sep="\t")
df["irr_pathway"] = (
    df["node_id"]
    .str.split(":")
    .apply(lambda x: x[1])
    .isin(irr_pathways_df["Pathway identifier"])
)
df.to_parquet("enriched_openbiolink_df.parquet", index=False)

/nfs_home/users/thzo/aNNGEL/anngel/datasets/custom_dataset/__init__.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(self.edges_file, sep="\t")


Only local copy available.
Only local copy available.


/nfs_home/users/thzo/aNNGEL/anngel/datasets/custom_dataset/__init__.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(self.edges_file, sep="\t")


Only local copy available.
Only local copy available.


In [3]:
irr_pathway_idcs = np.where(df["irr_pathway"])[0]
gene_idcs = np.where(df["node_type"] == "GENE")[0]
pairs_pathway = [(a, b) for a in gene_idcs for b in irr_pathway_idcs]

with open("pairs_obl_enriched_pathway.pkl", "wb") as f:
    pickle.dump(pairs_pathway, f)
# pairs_pathway_dsf.upload()

sources = np.sort(np.unique(np.array(pairs_pathway)[:, 0]))
targets = np.sort(np.unique(np.array(pairs_pathway)[:, 1]))
input_pairs_pathway = [(g_idx, targets) for g_idx in sources]
with open("input_pairs_obl_enriched_pathway.pkl", "wb") as f:
    pickle.dump(input_pairs_pathway, f)
# input_pairs_pathway_dsf.upload()
del pairs_pathway
del input_pairs_pathway

Only local copy available.
Only local copy available.


In [5]:
ds.get_nodes().to_csv("nodes_obl_enriched.csv", index=False)

Only local copy available.
